## 作業

請參考昨天的程式碼，將訓練資料讀取方式改寫成 Generator，並將原本的 model.fit 改為 model.fit_generator 來進行訓練。請參考 Keras [官方文件中 fit_generator 的說明](https://keras.io/models/sequential/)

In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

# 防止調用 GPU 報錯：Blas GEMM launch failed
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.8 #佔用 GPU 30% 的記憶體資源
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
# 超參數設定
BATCH_SIZE = [256, 512] # batch 的大小，如果出現 OOM error，請降低這個值
EPOCHS = [12, 18, 20] # 訓練的 epochs 數量

In [3]:
def generate_batch_data(x_train, y_train, bs):
    while True:
        for indexs in range(0, len(x_train), bs):
            sub_x_train = x_train[indexs: indexs+bs]
            sub_y_train = y_train[indexs: indexs+bs]
            yield sub_x_train, sub_y_train

## DNN

In [4]:
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
# 將資料攤平成一維資料
x_train = x_train.reshape(50000, 3072) 
x_test = x_test.reshape(10000, 3072)

# 將資料變為 float32 並標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(3072,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

for bs in BATCH_SIZE:
    for ep in EPOCHS:
        history = model.fit_generator(generate_batch_data(x_train, y_train, bs),
                                      steps_per_epoch=len(y_train)//bs,
                                      epochs=ep,
                                      verbose=1,
                                      validation_data=(x_test, y_test))
        score = model.evaluate(x_test, y_test, verbose=0)
        print('--------------------------------------------------\nbs = ',bs,', ep=',ep)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        print('\n\n')

50000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________
Epoch 1/12
195/195 [==============================] - 2s 11ms/step - loss: 2.4561 - acc: 0.22

97/97 [==============================] - 1s 12ms/step - loss: 1.2935 - acc: 0.5379 - val_loss: 1.4660 - val_acc: 0.4833
Epoch 3/12
97/97 [==============================] - 1s 9ms/step - loss: 1.2861 - acc: 0.5389 - val_loss: 1.3780 - val_acc: 0.5116
Epoch 4/12
97/97 [==============================] - 1s 9ms/step - loss: 1.2843 - acc: 0.5414 - val_loss: 1.3753 - val_acc: 0.5139
Epoch 5/12
97/97 [==============================] - 1s 10ms/step - loss: 1.2764 - acc: 0.5447 - val_loss: 1.3864 - val_acc: 0.5105
Epoch 6/12
97/97 [==============================] - 1s 12ms/step - loss: 1.2756 - acc: 0.5440 - val_loss: 1.3772 - val_acc: 0.5170
Epoch 7/12
97/97 [==============================] - 1s 9ms/step - loss: 1.2769 - acc: 0.5432 - val_loss: 1.3662 - val_acc: 0.5142
Epoch 8/12
97/97 [==============================] - 1s 9ms/step - loss: 1.2754 - acc: 0.5440 - val_loss: 1.3932 - val_acc: 0.5109
Epoch 9/12
97/97 [==============================] - 1s 9ms/step - loss: 1.2656 - acc: 0.5478 - val

## CNN

In [6]:
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

for bs in BATCH_SIZE:
    for ep in EPOCHS:
        history = model.fit_generator(generate_batch_data(x_train, y_train, bs),
                                      steps_per_epoch=len(y_train)//bs,
                                      epochs=ep,
                                      verbose=1,
                                      validation_data=(x_test, y_test))
        score = model.evaluate(x_test, y_test, verbose=0)
        print('--------------------------------------------------\nbs = ',bs,', ep=',ep)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        print('\n\n')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_14 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 64)        18496     
__________

195/195 [==============================] - 6s 33ms/step - loss: 0.4134 - acc: 0.8637 - val_loss: 0.7233 - val_acc: 0.7750
Epoch 10/20
195/195 [==============================] - 7s 34ms/step - loss: 0.4145 - acc: 0.8619 - val_loss: 0.8183 - val_acc: 0.7880
Epoch 11/20
195/195 [==============================] - 7s 35ms/step - loss: 0.4147 - acc: 0.8613 - val_loss: 0.8937 - val_acc: 0.7966
Epoch 12/20
195/195 [==============================] - 7s 36ms/step - loss: 0.4195 - acc: 0.8617 - val_loss: 0.6970 - val_acc: 0.7894
Epoch 13/20
195/195 [==============================] - 7s 35ms/step - loss: 0.4165 - acc: 0.8645 - val_loss: 0.7921 - val_acc: 0.7989
Epoch 14/20
195/195 [==============================] - 7s 33ms/step - loss: 0.4135 - acc: 0.8628 - val_loss: 0.6427 - val_acc: 0.7949
Epoch 15/20
195/195 [==============================] - 6s 33ms/step - loss: 0.4122 - acc: 0.8642 - val_loss: 0.6222 - val_acc: 0.8034
Epoch 16/20
195/195 [==============================] - 7s 34ms/step - loss

97/97 [==============================] - 6s 62ms/step - loss: 0.2811 - acc: 0.9105 - val_loss: 0.6895 - val_acc: 0.8034
Epoch 19/20
97/97 [==============================] - 6s 62ms/step - loss: 0.2833 - acc: 0.9099 - val_loss: 0.9979 - val_acc: 0.8059
Epoch 20/20
97/97 [==============================] - 6s 61ms/step - loss: 0.2785 - acc: 0.9109 - val_loss: 0.8040 - val_acc: 0.8114
--------------------------------------------------
bs =  512 , ep= 20
Test loss: 0.8039754345655441
Test accuracy: 0.8114



